<a href="https://colab.research.google.com/github/AIFahim/AIFahim/blob/main/Training_With_Glove_Model_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
import gensim
import string

from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam

import plotly.express as px

import zipfile



import tensorflow.keras as keras 
import tensorflow as tf
import tensorflow_hub as hub



class color: # Text style
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Loading the data and getting besic information about data

In [ ]:
# Download Dataset
!gdown --id 1boKJGs-XUWsxz6JocgU7wyhf8nSayr7k # train csv
!gdown --id 1WLTPLk8oJ0XDduXnzLmxM3xwlChsgs81 # test csv
!gdown --id 1-6MlDGVEZlLFQliVL6Hgxee1wnpi_yeG # full data csv after added new clean text column

Downloading...
From: https://drive.google.com/uc?id=1boKJGs-XUWsxz6JocgU7wyhf8nSayr7k
To: /content/train.csv
100% 859M/859M [00:04<00:00, 198MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WLTPLk8oJ0XDduXnzLmxM3xwlChsgs81
To: /content/test.csv
100% 854M/854M [00:03<00:00, 246MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-6MlDGVEZlLFQliVL6Hgxee1wnpi_yeG
To: /content/fulldata.csv
100% 2.39G/2.39G [00:19<00:00, 125MB/s]


In [ ]:
train= pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
fulldata = pd.read_csv("/content/fulldata.csv")

In [ ]:
fulldata.head()

,Unnamed: 0,category,text,cleanText,new_clean_text
0,0,education,"অধ্যায়-২প্রিয় পরীক্ষার্থী, আজ তথ্য ও যোগাযোগ প...",অধ্যায় প্রিয় পরীক্ষার্থী তথ্য যো...,অধ্যায় প্রিয় পরীক্ষার্থী তথ্য যোগাযোগ প্রযুক্ত...
1,1,sports,"স্পিন, স্পিন, স্পিন। লর্ডসে ইয়াসির শাহ্‌র লেগ ...",স্পিন স্পিন স্পিন লর্ডসে ইয়াসির ...,স্পিন স্পিন স্পিন লর্ডসে ইয়াসির শাহ্ লেগ স্পিন...
2,2,international,সিরিয়ার উত্তর-পশ্চিমাঞ্চলে গতকাল রোববার বিমান...,সিরিয়ার উত্তর পশ্চিমাঞ্চলে গতকাল ...,সিরিয়ার পশ্চিমাঞ্চলে গতকাল রোববার বিমান হামলা...
3,3,international,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যালয়ের (ইউসিট...,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যা...,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যালয়ের ইউসিটি...
4,4,technology,সহজে ডিজিটাল আর্থিক সেবা দিতে অ্যাপ ও ডিজিটাল ...,সহজে ডিজিটাল আর্থিক সেবা অ্যাপ ...,সহজে ডিজিটাল আর্থিক সেবা অ্যাপ ডিজিটাল কেওয়াইস...


In [ ]:
fulldata.columns

Index(['Unnamed: 0', 'category', 'text', 'cleanText', 'new_clean_text'], dtype='object')

In [ ]:
fulldata.pop('Unnamed: 0') # Remove unexpected column

0             0
1             1
2             2
3             3
4             4
          ...  
197762    98878
197763    98879
197764    98880
197765    98881
197766    98882
Name: Unnamed: 0, Length: 197767, dtype: int64

In [ ]:
fulldata.head()

,category,text,cleanText,new_clean_text
0,education,"অধ্যায়-২প্রিয় পরীক্ষার্থী, আজ তথ্য ও যোগাযোগ প...",অধ্যায় প্রিয় পরীক্ষার্থী তথ্য যো...,অধ্যায় প্রিয় পরীক্ষার্থী তথ্য যোগাযোগ প্রযুক্ত...
1,sports,"স্পিন, স্পিন, স্পিন। লর্ডসে ইয়াসির শাহ্‌র লেগ ...",স্পিন স্পিন স্পিন লর্ডসে ইয়াসির ...,স্পিন স্পিন স্পিন লর্ডসে ইয়াসির শাহ্ লেগ স্পিন...
2,international,সিরিয়ার উত্তর-পশ্চিমাঞ্চলে গতকাল রোববার বিমান...,সিরিয়ার উত্তর পশ্চিমাঞ্চলে গতকাল ...,সিরিয়ার পশ্চিমাঞ্চলে গতকাল রোববার বিমান হামলা...
3,international,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যালয়ের (ইউসিট...,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যা...,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যালয়ের ইউসিটি...
4,technology,সহজে ডিজিটাল আর্থিক সেবা দিতে অ্যাপ ও ডিজিটাল ...,সহজে ডিজিটাল আর্থিক সেবা অ্যাপ ...,সহজে ডিজিটাল আর্থিক সেবা অ্যাপ ডিজিটাল কেওয়াইস...


## Preprocessing

### Convert Categories String to  Categories Number 

In [ ]:
def label_encoding(dataset,bool):
  '''
  This function will convert categorical class names into a numeric label

  Args:
  dataset: A dataset
  bool : a boolean (True or False)

  returns:
  labels: encoded labels 
  '''
  le = LabelEncoder()
  le.fit(dataset.category)
  encoded_labels = le.transform(dataset.category)
  labels = np.array(encoded_labels) # Converting into numpy array
  class_names =le.classes_ ## Define the class names again
  if bool == True:
    print(color.BOLD+"\n\t\t\t===== Label Encoding ====="+color.END,"\nClass Names:-->",le.classes_)
  return labels 

In [ ]:
labels = label_encoding(fulldata,True)


			===== Label Encoding ===== 
Class Names:--> ['economy' 'education' 'entertainment' 'international' 'sports' 'state'
 'technology']


### Glove for Vectorization

Use Glove pretrained corpus model to represent words.

In [ ]:
!gdown --id 1hau9l8myQOVF2cR4GBPeCtUDi5UOk1_P # Download Bengali GloVe Word Vectors

Downloading...
From: https://drive.google.com/uc?id=1hau9l8myQOVF2cR4GBPeCtUDi5UOk1_P
To: /content/bn_glove.39M.100d.txt
100% 173M/173M [00:00<00:00, 212MB/s]


In [ ]:
# import and define Blnp for tokenizer
!pip install bnlp_toolkit
from bnlp import NLTKTokenizer 
bnltk = NLTKTokenizer()

     |████████████████████████████████| 23.9 MB 1.1 MB/s 
     |████████████████████████████████| 1.2 MB 63.0 MB/s 
     |████████████████████████████████| 743 kB 67.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


punkt not found. downloading...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from bnlp.corpus import stopwords

def create_corpus(df):
    """
    This function will return the tokenizer of using bnltk tokenizer. 
    
    Args:
        df: dataset corpus
        
    Returns:
        corpus: tokenized corpus 
        
    """

    corpus=[]
    for train in tqdm(df['new_clean_text']):
        words=[word for word in bnltk.word_tokenize(train) if( (word not in stopwords))]    
        corpus.append(words)
    return corpus     

In [ ]:
corpus=create_corpus(fulldata)

100%|██████████| 197767/197767 [07:36<00:00, 432.97it/s]


In [ ]:
# Check corpus

for i,_ in enumerate(corpus):
  print(_)
  if i == 5:
    break

['অধ্যায়', 'প্রিয়', 'পরীক্ষার্থী', 'তথ্য', 'যোগাযোগ', 'প্রযুক্তি', 'বিষয়ের', 'সৃজনশীল', 'নমুনা', 'প্রশ্নোত্তর', 'করিম', 'সাহেব', 'একজন', 'ব্যবসায়ী', 'ছোট', 'ভাই', 'মাসুদকে', 'অফিসের', 'কম্পিউটারগুলো', 'পরস্পরের', 'সংযোগের', 'কেব্ল', 'অর্থসাশ্রয়ী', 'সহজে', 'স্থাপনযোগ্য', 'ছোট', 'ভাই', 'মাসুদের', 'অফিসে', 'দেখলেন', 'অফিসে', 'সেদিন', 'কম্পিউটার', 'অনুসন্ধানে', 'কম্পিউটারের', 'সংযোগ', 'কেব্ল', 'নষ্ট', 'পরের', 'ভাইয়ের', 'অফিসে', 'দেখলেন', 'অফিসের', 'কম্পিউটার', 'কেন্দ্রীয়', 'ডিভাইসের', 'সংযুক্ত', 'মডেম', 'ট্রান্সমিশন', 'মোডে', 'উভয়', 'ডেটা', 'আদান', 'প্রদান', 'ব্যাখ্যা', 'করো', 'করিম', 'সাহেবের', 'অফিসে', 'ব্যবহূত', 'কেব্লটি', 'ধরনের', 'ব্যাখ্যা', 'করো', 'নেটওয়ার্ক', 'সংযোগের', 'উদ্দীপকে', 'উল্লিখিত', 'ব্যবস্থাদ্বয়ের', 'কোনটি', 'উত্তম', 'বিশ্লেষণপূর্বক', 'মতামত', 'দাও', 'কমডেম', 'সংক্ষিপ্ত', 'রূপ', 'মডেম', 'ইলেকট্রনিক', 'ডিভাইস', 'অ্যানালগ', 'সংকেতকে', 'ডিজিটাল', 'ডিজিটাল', 'সংকেতকে', 'অ্যানালগ', 'সংকেতে', 'রূপান্তর', 'খফুল', 'ডুপ্লেক্স', 'মোডে', 'উভয়', 'ডেটা', 'আদান', 'প্রদান', 'ডেটা', 'ট্

In [ ]:
embedding_dict={}
with open('/content/bn_glove.39M.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

train_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
trainer=train_pad[:train.shape[0]]

test=train_pad[train.shape[0]:]

labels_train = labels[:train.shape[0]]
lables_test = labels[train.shape[0]:]

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 677924


In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

100%|██████████| 677924/677924 [00:00<00:00, 841047.27it/s]


## Tpu Stretigy Define

In [ ]:
## TPU Stretigy 

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.89.45.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.89.45.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Model Define

In [ ]:
K.clear_session()
with strategy.scope():
  
  model=Sequential()

  embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                    input_length=MAX_LEN,trainable=False)

  model.add(embedding)
  model.add(SpatialDropout1D(0.2))
  model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(7))

  def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

  def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
  
  def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
  

  optimzer=Adam(learning_rate=1e-5)

  model.compile(loss='sparse_categorical_crossentropy',optimizer=optimzer,metrics=['accuracy',recall_m,precision_m,f1_m])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           67792500  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 50, 100)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 7)                 455       
Total params: 67,835,195
Trainable params: 42,695
Non-trainable params: 67,792,500
_________________________________________________________________


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(trainer,labels_train,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (84050, 50)
Shape of Validation  (14833, 50)


In [ ]:
from google.colab import drive
drive.mount('/content/drive') ## Mount Google Drive

Mounted at /content/drive


In [ ]:
# checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Callbacks Glove Model Ep 39 - 59/model39_59.h5', monitor="val_loss", save_best_only=True)
# checkpoint1 = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/Callbacks Glove Model Ep 39 - 59/History39_59.csv')
# history = model.fit(
#                   X_train,y_train,
#                   batch_size=16,
#                   epochs=20,
#                   callbacks=[checkpoint, checkpoint1],
#                   validation_data=(X_test,y_test)
# )

In [ ]:
## Load and Train Again

model.load_weights("/content/drive/MyDrive/Callbacks Glove Model Ep 21 - 80/model21_80.h5")

checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Callbacks Glove Model Ep 81 - further/model81_further.h5', monitor="val_loss", save_best_only=True)
checkpoint1 = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/Callbacks Glove Model Ep 81 - further/History81_further.csv')


import gc
# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()


history = model.fit(
                  X_train,y_train,
                  batch_size=16,
                  epochs=60,
                  callbacks=[checkpoint, checkpoint1, GarbageCollectorCallback()],
                  validation_data=(X_test,y_test),

)

Epoch 1/60
   4/5254 [..............................] - ETA: 2:10 - loss: 1.0156 - accuracy: 0.8125 - recall_m: 1.7812 - precision_m: 1.3255 - f1_m: 1.4813WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_train_batch_end` time: 0.0288s). Check your callbacks.


5254/5254 [==============================] - 128s 24ms/step - loss: 0.8318 - accuracy: 0.8122 - recall_m: 1.7323 - precision_m: 3569.9480 - f1_m: 1.4186 - val_loss: 0.6730 - val_accuracy: 0.8468 - val_recall_m: 1.5981 - val_precision_m: 1.1726 - val_f1_m: 1.3025
Epoch 2/60
5254/5254 [==============================] - 132s 25ms/step - loss: 0.8343 - accuracy: 0.8136 - recall_m: 1.7514 - precision_m: 6662.7700 - f1_m: 1.4280 - val_loss: 0.6686 - val_accuracy: 0.8422 - val_recall_m: 1.6453 - val_precision_m: 1.1946 - val_f1_m: 1.3321
Epoch 3/60
5254/5254 [==============================] - 132s 25ms/step - loss: 0.8332 - accuracy: 0.8122 - recall_m: 1.7612 - precision_m: 9041.7383 - f1_m: 1.4367 - val_loss: 0.6666 - val_accuracy: 0.8436 - val_recall_m: 1.6340 - val_precision_m: 1.1922 - val_f1_m: 1.3280
Epoch 4/60
5254/5254 [==============================] - 133s 25ms/step - loss: 0.8135 - accuracy: 0.8140 - recall_m: 1.7373 - precision_m: 2856.2319 - f1_m: 1.4239 - val_loss: 0.6665 - val_

IndexError: ignored

In [ ]:
gc.collect()